In [ ]:
!pip install textract
!pip install glob2

In [ ]:
#필요한 모듈 import
import re
import textract
import glob
import os

In [ ]:
#전처리 함수
def PreProcess(text):
    text = re.sub(pattern='Posted on [0-9]{4} [0-9]{2} [0-9]{2}', \
                  repl='', string=text)
    text = re.sub(pattern='Posted on [0-9]{8}', \
                  repl='', string=text)
    text = re.sub(pattern='[0-9]{4}년 [0-9]{,2}월 [0-9]{,2}일 [0-9]{,2}시 [0-9]{,2}분 [0-9]{,2}초', \
                  repl='', string=text)
    text = re.sub(pattern='[0-9]{4}. [0-9]{,2}. [0-9]{,2}', \
                  repl='', string=text)
    _filter = re.compile('[\s]')
    text = _filter.sub(' ', text)
    
    _filter = re.compile('[ㄱ-ㅣ]+')
    text = _filter.sub('', text)
    
    _filter = re.compile('[\n]')
    text = _filter.sub(' ', text)
    
    _filter = re.compile('[^가-힣 0-9 a-z A-Z \. \, \' \" \? \!]+')
    text = _filter.sub('', text)
      
    
    #text = re.sub("[.]", ".  ", text).strip()

    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

## 주의
ANSI로 인코딩된 txt 파일이어야 정상적으로 preprocessing 되는 것을 확인했음


In [ ]:
#txt file 불러오기
#num = 몇 호인지 체크하기 위함 

#사용자 지정
num = 28
end_num = 38
processed_folder_path = 'C:/Users/user/Desktop/TAVE/북커톤/data'

#while num <= end_num:
    
path = 'C:/Users/user/Desktop/TAVE/북커톤/data/literature_repreprocess.txt'
files = [f for f in glob.glob(path)]

text_list = []

for file in files:
    with open(path, 'r', encoding = 'utf8') as f:
        raw_data = f.read()
        if raw_data == '':
            continue
    text_list.append(raw_data)

to_train_text = ''.join(map(str, text_list))
preprocessed_text = PreProcess(to_train_text)

processed_file_path = os.path.join(processed_folder_path, "xml_병합3.txt")
with open(processed_file_path, 'w', encoding = 'utf8') as f:
        f.write(preprocessed_text) 

    #num += 1

<h2>파일 구조</h2>

path = Desktop/TAVE/북커톤/data/{num}호

책자 수록된 수필을 모은 데이터이므로 각 데이터를 호수 별로 정리해 놨었다.

2호에서 27호까지 존재

따라서 해당 경로를 while문을 통해 접근하여 불러오고 한꺼번에 파일을 합쳐 전처리 한다.

참고: https://github.com/jeina7/GPT2-essay-writer/blob/main/notebooks/03.%20%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0.ipynb


## 맞춤법 검사기

In [ ]:
import pandas as pd
import os
from hanspell import spell_checker
from tqdm import tqdm

# pp_df는 전처리 된 데이터
sc_df = pd.DataFrame({'context':[]})

data_dir = 'C:/Users/sally/Desktop/북커톤/preprocess2_대표에세이 문학회/repreprocessing'
save_dir = 'C:/Users/sally/Desktop/북커톤/preprocess2_대표에세이 문학회'

for text_name in tqdm(sorted(os.listdir(data_dir))):
    text_dir = os.path.join(data_dir, text_name)
    with open(text_dir, 'r') as f:
        raw_texts = f.readlines()
        if len(raw_texts) == 0:
            continue
        
        output_text = ''
        for raw_text in raw_texts:
            preprocessed_text = preprocessing2(raw_text)
            try:
                spell_checked_text = spell_checker.check(preprocessed_text)
                spell_checked_text = spell_checked_text.as_dict()['checked']
                if spell_checked_text and spell_checked_text[0] != ' ':
                    spell_checked_text = ' ' + spell_checked_text
            except:
                continue
            output_text = output_text + spell_checked_text
        sc_df = sc_df.append({'context': output_text}, ignore_index=True)

sc_df.head(10)
sc_df.to_csv(os.path.join(save_dir, 'essay_sc.txt'), sep='\n', header=False, index=False)
with open(os.path.join(save_dir, 'essay_sc_not_df.txt'), 'w') as f:
    for i, text in enumerate(sc_df['context']):
        if i != 0:
            f.write('\n'+text)
        else:
            f.write(text)

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: Future